Мы определяем следующий пул компаний для который будем считать Information Ratio для компаний:
* S&P 500 (^GSPC) - наш бенчмарк
* FORD (F)
* Amazon (AMZN)
* Tesla (TSLA)
* Apple (AAPL)
* Yandex (YNDX)
* NIVIDA(NVDA)
* Airbnb (ABNB)
* CocaCola Consolidated (COKE)
* Netflix (NFLX)


Сначала необходимо определить откуда мы будем брать информацию. Наша команда выбрала сайт Yahoo.com для импорта данных о ценных бумагах компаний, представленных выше.

In [8]:
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import yfinance as yfin
yfin.pdr_override()

ImportError: cannot import name 'PANDAS_0230' from 'pandas_datareader.compat' (c:\Python\Lib\site-packages\pandas_datareader\compat\__init__.py)

In [4]:
def get_data(name:str):
    '''
    Эта функция импортирует данные по названию компании
    '''
    data = pdr.get_data_yahoo([name], start='2019-1-1', end='2022-12-31')['Adj Close'].reset_index()
    data = pd.DataFrame(data)
    data.rename(columns={'Adj Close':  f'{name}'}, inplace=True)
    return data

short_names = ['^GSPC','F','AMZN','TSLA', 'AAPL', 'YNDX', 'NVDA', 'ABNB', 'COKE', 'NFLX']
data = get_data(short_names[0])
for i in short_names:
    if i != '^GSPC':
        data = pd.merge(data, get_data(i), on = 'Date', how = 'inner')
        
data



NameError: name 'pdr' is not defined

Теперь необходимо посчитать Return для каждой компании

In [50]:
returns_daily = pd.DataFrame()
returns_daily['Date'] = data['Date']
for company in data.columns[1:]:
    returns_daily[company] = data[company].pct_change()

returns_daily.iloc[1::]

,Date,^GSPC,F,AMZN,TSLA,AAPL,YNDX,NVDA,ABNB,COKE,NFLX
1,2020-12-11,-0.001265,-0.010965,0.004814,-0.027238,-0.006735,-0.005659,0.003161,-0.037731,-0.001762,0.004251
2,2020-12-14,-0.004359,-0.012195,0.013012,0.048919,-0.005147,-0.029169,0.022708,-0.066427,0.008713,0.038154
3,2020-12-15,0.012921,0.026936,0.002582,-0.010284,0.050090,0.009673,0.003888,-0.040000,0.011872,-0.005053
4,2020-12-16,0.001773,-0.012022,0.023961,-0.016550,-0.000547,0.027725,-0.008832,0.105689,0.004324,0.009716
5,2020-12-17,0.005758,0.004425,-0.001506,0.053198,0.006964,-0.002119,0.007457,0.065657,0.005616,0.015376
...,...,...,...,...,...,...,...,...,...,...,...
513,2022-12-23,0.005868,0.004421,0.017425,-0.017551,-0.002798,0.000000,-0.008671,0.004477,0.016330,-0.009370
514,2022-12-27,-0.004050,-0.014084,-0.025924,-0.114089,-0.013878,0.000000,-0.071353,-0.020645,-0.008646,-0.036581
515,2022-12-28,-0.012021,-0.022321,-0.014692,0.033089,-0.030685,0.000000,-0.006020,-0.011977,-0.017828,-0.025654
516,2022-12-29,0.017461,0.053881,0.028844,0.080827,0.028324,0.000000,0.040396,0.033216,0.021589,0.051430


Теперь необходимо посчитать Return каждой за каждый год: 2020, 2021, 2022.

In [1]:
returns_2021 = returns_daily[returns_daily['Date'] <  2021-1-1]
returns_2021

NameError: name 'returns_daily' is not defined

In [64]:
returns_for_period = dict()

for company in returns_daily.columns[1:]:
    returns_for_period[company] = np.prod(returns_daily[company] + 1) - 1
    
returns_for_period

{'^GSPC': 0.04672716059555526,
 'F': 0.32342512246858934,
 'AMZN': -0.4583248466342773,
 'TSLA': -0.4106877930829641,
 'AAPL': 0.06694010498999559,
 'YNDX': -0.7320316857345757,
 'NVDA': 0.12859674438293278,
 'ABNB': -0.40916318130605867,
 'COKE': 0.9728973796324165,
 'NFLX': -0.41152286607619404}

Сейчас необходимо посчитать Tracking Error для каждой компании.

In [65]:
# Сначала необходимо посчитать разницу в Return бенчмарка и Return компаний
diff_returns = pd.DataFrame()
for company in returns_daily.columns[2:]:
    diff_returns[f'{company}_diff'] = returns_daily[company] - returns_daily['^GSPC']
    
diff_returns

,F_diff,AMZN_diff,TSLA_diff,AAPL_diff,YNDX_diff,NVDA_diff,ABNB_diff,COKE_diff,NFLX_diff
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.009700,0.006079,-0.025973,-0.005470,-0.004394,0.004426,-0.036466,-0.000497,0.005516
2,-0.007836,0.017371,0.053278,-0.000787,-0.024810,0.027067,-0.062068,0.013072,0.042514
3,0.014015,-0.010340,-0.023205,0.037169,-0.003248,-0.009033,-0.052921,-0.001050,-0.017975
4,-0.013794,0.022188,-0.018322,-0.002320,0.025953,-0.010605,0.103916,0.002552,0.007943
...,...,...,...,...,...,...,...,...,...
513,-0.001447,0.011556,-0.023419,-0.008666,-0.005868,-0.014539,-0.001391,0.010461,-0.015238
514,-0.010035,-0.021874,-0.110039,-0.009829,0.004050,-0.067304,-0.016596,-0.004596,-0.032532
515,-0.010301,-0.002671,0.045110,-0.018665,0.012021,0.006001,0.000043,-0.005807,-0.013633
516,0.036420,0.011382,0.063366,0.010863,-0.017461,0.022935,0.015755,0.004128,0.033969


In [72]:
tracking_errors = dict()

for company in diff_returns.columns[2:]:
    tracking_errors[company[:4]] = diff_returns[company].std() * np.sqrt(252)
tracking_errors

{'TSLA': 0.5259042119887413,
 'AAPL': 0.18711454224715246,
 'YNDX': 0.45159066022846,
 'NVDA': 0.4147933826521154,
 'ABNB': 0.5005079577144395,
 'COKE': 0.3766595463202601,
 'NFLX': 0.46451053050732527}

Теперь считаем сам Information Ratio для каждой компании

In [74]:
information_ratio = dict()

for company in tracking_errors.keys():
    information_ratio[company] = (returns_for_period[company] - returns_for_period['^GSPC']) / tracking_errors[company]
    
information_ratio

{'TSLA': -0.8697685685930804,
 'AAPL': 0.10802444402072084,
 'YNDX': -1.7244795229736511,
 'NVDA': 0.1973743729080679,
 'ABNB': -0.9108553318181571,
 'COKE': 2.458905470696266,
 'NFLX': -0.9865223640274884}